# Merge user and session data

- We add 'action' activities using word vectors by counting the number of each action. 
  (like word counting vector from text analysis)
- We also keep the total second for each action by creating another 360-dim word vector of total seconds for each action. 
- Therefore, we add 720-dim feature vector for each user. 
- Finally, we have 650Kx720-dim DataFrame and save it for modeling. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('train_users_2.csv')
df_s = pd.read_csv('sessions.csv')

In [3]:
def get_language(s):
    if s == 'en':
        return s
    else:
        return 'not en'

def get_device(s):
    if s in ['Mac Desktop', 'Windows Desktop', 'iPhone', 'iPad']:
        return s.split(' ')[0]
    else:
        return 'Else'

def get_affiliate_provider(s):
    if s in ['direct', 'google']:
        return s
    else:
        return 'other'
    
def get_browser(s):
    if s in ['Chrome', 'Safari', 'Firefox', 'IE', '-unknown-', 'Android Browser', 'Mobile Safari', 'Chrome Mobile']:
        return s
    else:
        return 'Else'

def timedelta_to_day(time_delta):
    if pd.notnull(time_delta):
        return time_delta.days
    return np.nan

def preprocess(df_origin):
    df = df_origin.copy()
    df = df[df.country_destination.isin(['NDF', 'US'])]
    
    df['language'] = df['language'].apply(get_language)
    df['first_device_type'] = df['first_device_type'].apply(get_device)
    df['affiliate_provider'] = df['affiliate_provider'].apply(get_affiliate_provider)
    df['first_browser'] = df['first_browser'].apply(get_browser)
    
    df.age.fillna(1000, inplace=True)
    df['ageCat'] = pd.cut(df['age'], 
                            bins=[0,10,20,30,40,50,60,70,80,90,100,10000])

    df['date_account_created'] = pd.to_datetime(df['date_account_created'], format='%Y-%m-%d')
    df['date_first_booking'] = pd.to_datetime(df['date_first_booking'], format='%Y-%m-%d')
    df['create_year'] = df.date_account_created.apply(lambda x: x.year)
    df['create_month'] = df.date_account_created.apply(lambda x: x.month)
    df['time_create_to_book'] = (df['date_first_booking']-df['date_account_created']).apply(timedelta_to_day)
    df = df[~(df.time_create_to_book < 0)]
    df['time_create_to_book_cat'] = pd.cut(df['time_create_to_book'], 
                        bins=[-0.1,1, 7, 30, 90,1000])
    
    return df

In [4]:
df = preprocess(df)

In [5]:
df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,ageCat,create_year,create_month,time_create_to_book,time_create_to_book_cat
0,gxn3p5htnn,2010-06-28,20090319043255,NaT,-unknown-,1000.0,facebook,0,en,direct,...,untracked,Web,Mac,Chrome,NDF,"(100, 10000]",2010,6,NaN,NaN
1,820tgsjxq7,2011-05-25,20090523174809,NaT,MALE,38.0,facebook,0,en,seo,...,untracked,Web,Mac,Chrome,NDF,"(30, 40]",2011,5,NaN,NaN
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,1000.0,basic,0,en,other,...,omg,Web,Mac,Chrome,US,"(100, 10000]",2010,1,1.0,"(-0.1, 1]"
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,...,untracked,Web,Mac,Safari,US,"(40, 50]",2010,1,3.0,"(1, 7]"
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,basic,0,en,direct,...,omg,Web,Mac,Safari,US,"(40, 50]",2010,1,10.0,"(7, 30]"


## ID analysis

In [6]:
df.shape

(186898, 21)

In [7]:
ids_with_session = df[df.id.isin(df_s.user_id)].id
ids_with_session.shape

(65136,)

- 35% of users in train data are having session data. 
- This is because sesseion data are collected for users for 2014. 
- We first only use user data with session data. 

In [8]:
df_test = pd.read_csv('test_users.csv')

In [9]:
df_test.shape

(62096, 15)

In [10]:
df_test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [11]:
ids_test_with_session = df_test[df_test.id.isin(df_s.user_id)].id
ids_test_with_session.shape

(61668,)

- 99.3% of test users have session data
- Therefore, we are safe to model using session data. 

In [12]:
df = df[df.id.isin(ids_with_session)]

In [13]:
df.shape

(65136, 21)

In [14]:
df_s = df_s[df_s.user_id.isin(ids_with_session)]

In [15]:
df_s.shape

(4733234, 6)

In [16]:
df.to_csv('train_users_with_session.csv', index=False)
df_s.to_csv('train_sessions.csv', index=False)

# Combine User and Session Data

In [17]:
df = pd.read_csv('train_users_with_session.csv')
df_s = pd.read_csv('train_sessions.csv')

In [18]:
user_info = ['gender', 'ageCat','signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'signup_app', 'first_device_type', 'first_browser']
target = 'country_destination'

In [19]:
from sklearn.preprocessing import LabelEncoder
for col in user_info+[target]:
    df[col] = LabelEncoder().fit_transform(df[col].values)

In [20]:
import pyprind

def combineData(df_origin, df_s):
    
    df = df_origin.copy()
    
    col_name='action'
    col_idx = 1
    
    # extract unique action values 
    actions = [x if pd.notnull(x) else 'missing' for x in df_s[col_name].unique()]
    
    # add columns for each action values
    # 'bag-of-word' type model, with counting vectorization. 
    for x in actions:
        df[x] = 0
        df[x+'_sec'] = 0
        
    # for speed-up purpose, we use numpy values. 
    # compare to df.iterrows()
    df_v = df.values
    df_s_v = df_s.values

    hash_id_idx = {user_id:idx for idx, user_id in enumerate(df_v[:,0])}
    hash_action_idx = {col:idx for idx, col in enumerate(df.columns) if col in actions}
    
    n = df_s_v.shape[0]
    pbar = pyprind.ProgBar(n)
    for i in range(n):
        user_id = df_s_v[i,0]
        user_action = df_s_v[i,col_idx]
        user_action_sec = df_s_v[i,5]

        if pd.isnull(user_action):
            user_action = 'missing'
        if pd.isnull(user_action_sec):
            user_action_sec = 0

        user_idx = hash_id_idx[user_id]
        action_idx = hash_action_idx[user_action]
        df_v[user_idx, action_idx] += 1
        df_v[user_idx, action_idx+1] += user_action_sec

        pbar.update()
    
    df = pd.DataFrame(df_v, columns=df.columns)
    return df, actions

In [21]:
df2, actions = combineData(df, df_s)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:23


In [22]:
df2.shape

(65136, 679)

In [23]:
df2.iloc[0:10,21:29]

,dashboard,dashboard_sec,create,create_sec,confirm_email,confirm_email_sec,show,show_sec
0,1,2739,1,0,1,115983,3,71673
1,0,0,4,617933,0,0,0,0
2,0,0,2,12952,1,274002,15,562785
3,0,0,1,0,1,1.37162e+06,0,0
4,1,1355,0,0,1,46262,0,0
5,0,0,1,0,0,0,14,242969
6,1,1832,1,8834,2,11658,81,337983
7,1,1136,0,0,1,342501,0,0
8,0,0,1,0,0,0,16,190245
9,0,0,1,0,0,0,0,0


In [24]:
df2.to_csv('train_user_session_merged.csv', index=False)
pd.Series(actions).to_csv('actions.csv', index=False)